# Projeto: Pipeline de Dados e Análise com SQL no BigQuery

## 📖 Resumo e Introdução

**Projeto:** Pipeline de Dados e Análise com SQL no BigQuery – Livraria DevSaber

O nosso bjetivo é organizar os dados, analisar e gerar insights a partir dos dados de vendas da Livraria DevSaber, uma loja online fictícia.

O público-alvo desta documentação são pessoas que não possuem conhecimento prévio em BigQuery ou SQL, garantindo que qualquer leitor consiga compreender o fluxo de dados, análises e resultados.

**Escopo do Projeto:**
- Estruturar os dados em **tabelas normalizadas** (Clientes, Produtos, Vendas).
- Criar **views** para facilitar consultas.
- Realizar **análises** em SQL (RFM, Pareto, Co-purchase, Heatmaps, Tendências de Vendas).
- Produzir **relatórios visuais interativos**.
- Gerar documentação clara e detalhada do processo para fácil reutilização e entendimento futuro.

**Objetivo final:** Capacitar a empresa a extrair insights estratégicos sobre clientes, produtos e vendas de forma eficiente, simples e visual.


## 📑 Sumário do Notebook

1. [Resumo e Introdução](#Resumo-e-Introdução)
2. [Definição do Schema](#Definição-do-Schema)
3. [Ingestão de Dados](#Ingestão-de-Dados)
4. [Criação de Views](#Criação-de-Views)
5. [Análises](#Análises-Avançadas)
   - RFM
   - Pareto
   - Produtos Comprados Juntos
   - Receita por Mês
   - Heatmap de Vendas
   - Segmentação de Clientes
   - Previsão Simples
6. [Relatório Visual](#Relatório-Visual)
7. [Considerações Finais](#Considerações-Finais)


## 🔹 Descrição das Etapas

Neste projeto seguimos as seguintes etapas:

1. **Definição do Schema:** Criamos tabelas normalizadas para Clientes, Produtos e Vendas, para garantir que dados duplicados fossem evitados e mantendo integridade.

2. **Ingestão de Dados:** Inserimos os dados fornecidos na Livraria DevSaber usando SQL (`INSERT INTO`) no BigQuery.

3. **Criação de Views:** Construímos views (`vw_vendas_detalhes` e `vw_resumo_vendas`) para simplificar consultas complexas e gerar métricas agregadas.

4. **Análises:** Aplicamos SQL para extrair insights estratégicos sobre clientes, produtos e vendas.

5. **Relatório Visual:**


## 🔹 Definição do Schema

Neste bloco criamos as tabelas normalizadas para evitar duplicação de dados.

**Por que normalizamos?**
- Evita repetição de informações (como nome e email de clientes).
- Facilita atualizações futuras sem inconsistências.
- Mantém integridade de dados, essencial para análises corretas.

-- Tabela Clientes

```sql
    CREATE TABLE IF NOT EXISTS `Livraria_DevSaber_1_10.clientes` (
    cliente_id STRING,
    nome STRING,
    email STRING,
    estado STRING
    );

````

-- Tabela Produtos

```sql
    CREATE TABLE IF NOT EXISTS `Livraria_DevSaber_1_10.produtos` (
    produto_id STRING,
    nome_produto STRING,
    categoria STRING,
    preco NUMERIC
    );

````

-- Tabela Vendas


```sql
    CREATE TABLE IF NOT EXISTS `Livraria_DevSaber_1_10.vendas` (
    venda_id STRING,
    cliente_id STRING,
    produto_id STRING,
    quantidade INT64,
    data_venda DATE
    );



## 🔹 Ingestão de Dados


**Por que inserimos os dados?**
- Precisamos alimentar as tabelas para que possamos realizar análises concretas.
- Inserir os dados manualmente via `INSERT INTO` permite controle total sobre a integridade e unicidade das informações.


-- Inserindo clientes únicos

```sql
    INSERT INTO `Livraria_DevSaber_1_10.clientes` (cliente_id, nome, email, estado)
    VALUES
    ('C001', 'Ana Silva', 'ana.s@email.com', 'SP'),
    ('C002', 'Bruno Costa', 'b.costa@email.com', 'RJ'),
    ('C003', 'Carla Dias', 'carla.d@email.com', 'SP'),
    ('C004', 'Daniel Souza', 'daniel.s@email.com', 'MG');

````

-- Inserindo produtos


```sql
    INSERT INTO `Livraria_DevSaber_1_10.produtos` (produto_id, nome_produto, categoria, preco)
    VALUES
    ('P001','Fundamentos de SQL','Dados',60.00),
    ('P002','Duna','Ficção Cientifica',80.50),
    ('P003','Python para Dados','Programação',75.00),
    ('P004','O Guia do Mochileiro','Ficção Cientifica',42.00);

   ````

-- Inserindo vendas

```sql

    INSERT INTO `Livraria_DevSaber_1_10.vendas` (venda_id, cliente_id, produto_id, quantidade, data_venda)
    VALUES
    ('V001','C001','P001',1,'2024-01-15'),
    ('V002','C002','P002',1,'2024-01-18'),
    ('V003','C003','P003',1,'2024-02-02'),
    ('V004','C001','P002',1,'2024-02-10'),
    ('V005','C004','P001',1,'2024-02-20'),
    ('V006','C002','P004',1,'2024-03-05');


## 🔹 Criação de Views

As **Views** no BigQuery funcionam como consultas salvas.  
Em vez de escrever SQL complexo todas as vezes, basta consultar a View como se fosse uma tabela.


### ✅ Vantagens:
- **Reuso**: evita reescrever consultas longas e repetitivas.  
- **Padronização**: garante que todos os analistas usem a mesma lógica.  
- **Agilidade**: acelera as análises, facilitando a vida do time de dados e de negócio.  
- **Abstração**: o usuário final não precisa conhecer todos os detalhes da modelagem.  


### 📌 Exemplo de uso no projeto:

Criamos a View `Livraria_DevSaber_1_10.vw_vendas_detalhadas` para juntar **Clientes, Produtos e Vendas** em uma única estrutura. Ela une automaticamente os dados de **Clientes**, **Produtos** e **Vendas**, trazendo em uma única tabela todos os detalhes necessários para as análises do negócio.

```sql

    CREATE OR REPLACE VIEW `Livraria_DevSaber_1_10.vw_vendas_detalhes` AS
    SELECT

    v.venda_id,

    c.cliente_id,

    c.nome AS nome_cliente,

    c.email AS email_cliente,
  
    c.estado AS estado_cliente,

    p.produto_id,

    p.nome_produto,

    p.categoria AS categoria_produto,

    p.preco AS preco_produto,

    v.quantidade,

    (v.quantidade * p.preco) AS total_venda,

    v.data_venda

    FROM `Livraria_DevSaber_1_10.vendas` v

    JOIN `Livraria_DevSaber_1_10.clientes` c

    ON v.cliente_id = c.cliente_id

    JOIN `Livraria_DevSaber_1_10.produtos` p

    ON v.produto_id = p.produto_id;

`````
Assim, ao invés de escrever vários `JOINs`, basta rodar:

```sql
SELECT
  nome_cliente,
  estado_cliente,
  nome_produto,
  categoria_produto,
  preco_produto,
  data_venda
FROM `Livraria_DevSaber_1_10.vw_vendas_detalhadas`
WHERE estado_cliente = 'SP';

````

### Demais Views

*Essa VIEW já trará informações agregadas por cliente, produto, categoria e estado, facilitando criar dashboards*

```sql

CREATE OR REPLACE VIEW `Livraria_DevSaber_1_10.vw_resumo_vendas` AS
WITH resumo_cliente AS (
  SELECT
    nome_cliente,
    SUM(total_venda) AS total_gasto,
    COUNT(DISTINCT venda_id) AS num_compras,
    COUNT(DISTINCT produto_id) AS produtos_diferentes
  FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
  GROUP BY nome_cliente
),
resumo_produto AS (
  SELECT
    nome_produto,
    categoria_produto,
    SUM(quantidade) AS total_vendido,
    SUM(total_venda) AS receita_total
  FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
  GROUP BY nome_produto, categoria_produto
),
resumo_estado AS (
  SELECT
    estado_cliente,
    SUM(total_venda) AS receita_estado
  FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
  GROUP BY estado_cliente
)
SELECT
  rc.nome_cliente,
  rc.total_gasto,
  rc.num_compras,
  rc.produtos_diferentes,
  re.estado_cliente,
  rp.nome_produto,
  rp.categoria_produto,
  rp.total_vendido,
  rp.receita_total,
  re.receita_estado
FROM resumo_cliente rc
JOIN `Livraria_DevSaber_1_10.vw_vendas_detalhes` vd
  ON rc.nome_cliente = vd.nome_cliente
JOIN resumo_estado re
  ON vd.estado_cliente = re.estado_cliente
JOIN resumo_produto rp
  ON vd.nome_produto = rp.nome_produto;

````
➡️ A primeira view serve para análises detalhadas e cálculos personalizados.

➡️ A segunda view serve para relatórios rápidos e dashboards de gestão.


# 🔹 Análises

**Por que fazemos essas análises?**
- **RFM:** identifica clientes valiosos, auxiliando em marketing e fidelização.
- **Pareto:** mostra quais clientes ou produtos geram maior receita.
- **Co-purchase:** revela padrões de compra conjunta, útil para cross-selling.
- **Receita por mês:** monitora desempenho e tendências de vendas.
- **Heatmap de vendas:** identifica dias e horários de maior movimento.
- **Segmentação de clientes:** direciona estratégias de comunicação.
- **Previsão simples:** ajuda a planejar estoque e promoções futuras.

## 1. RFM

--Recência, Frequência e Valor

--**Objetivo:** Identificar clientes mais valiosos e engajados.

*Segmentar clientes (VIP, Regular, Ocasional), direcionar campanhas de marketing, aumentar fidelização e otimizar vendas.*

```sql

WITH rfm AS (
  SELECT
    cliente_id,
    nome_cliente,
    SUM(total_venda) AS valor_total,
    COUNT(venda_id) AS frequencia,
    DATE_DIFF(DATE("2024-03-31"), MAX(data_venda), DAY) AS recencia
  FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
  GROUP BY cliente_id, nome_cliente
)
SELECT *,
  CASE
    WHEN valor_total >= 150 THEN 'VIP'
    WHEN valor_total >= 75 THEN 'Regular'
    ELSE 'Ocasional'
  END AS segmento
FROM rfm
ORDER BY valor_total DESC;


    
````
   ## 2. Pareto
-- Clientes que geram 80% da receita

--**Objetivo:** Identificar os clientes mais importantes para a receita, seguindo a regra 80/20.

*Focar esforços de marketing e retenção nos clientes que realmente impactam a receita.*

```sql
    WITH receita_total AS (
    SELECT SUM(total_venda) AS receita_geral
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    ),
    clientes_receita AS (
    SELECT
    cliente_id,
    nome_cliente,
    SUM(total_venda) AS receita_cliente
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    GROUP BY cliente_id, nome_cliente
    ORDER BY receita_cliente DESC
    )
    SELECT
    c.*,
    r.receita_geral,
    SUM(c.receita_cliente) OVER (ORDER BY receita_cliente DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS receita_acumulada,
    (SUM(c.receita_cliente) OVER (ORDER BY receita_cliente DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)/r.receita_geral) AS percentual_acumulado
    FROM clientes_receita c
    CROSS JOIN receita_total r;

````

## 3. Co-purchase
 -- Produtos comprados juntos

 -- **Objetivo:** Identificar produtos frequentemente comprados pelo mesmo cliente.

 *Sugerir produtos complementares, aumentando vendas e ticket médio.*

```sql
    SELECT
    v1.produto_id AS produto_1,
    p1.nome_produto AS nome_produto_1,
    v2.produto_id AS produto_2,
    p2.nome_produto AS nome_produto_2,
    COUNT(DISTINCT v1.cliente_id) AS qtd_clientes_compraram_juntos
    FROM `Livraria_DevSaber_1_10.vendas` v1
    JOIN `Livraria_DevSaber_1_10.vendas` v2
    ON v1.cliente_id = v2.cliente_id AND v1.produto_id < v2.produto_id
    JOIN `Livraria_DevSaber_1_10.produtos` p1 ON v1.produto_id = p1.produto_id
    JOIN `Livraria_DevSaber_1_10.produtos` p2 ON v2.produto_id = p2.produto_id
    GROUP BY produto_1, nome_produto_1, produto_2, nome_produto_2
    ORDER BY qtd_clientes_compraram_juntos DESC;

````


## 4. Receita por mês
--**Objetivo:** Avaliar o desempenho das vendas ao longo do tempo, identificar tendências sazonais.

*Planejamento de estoque, promoções e campanhas de marketing de acordo com a demanda histórica.*

```sql
    SELECT
    FORMAT_DATE("%Y-%m", data_venda) AS ano_mes,
    SUM(total_venda) AS receita_total
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    GROUP BY ano_mes
    ORDER BY ano_mes;

````

## 5. Heatmap de vendas
--**Objetivo:** Identificar os dias da semana com maior volume de vendas.

*Planejar promoções e campanhas em dias estratégicos de maior movimento.*

```sql
    SELECT
    EXTRACT(DAYOFWEEK FROM data_venda) AS dia_semana,
    COUNT(venda_id) AS qtd_vendas,
    SUM(total_venda) AS receita
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    GROUP BY dia_semana
    ORDER BY dia_semana;

````

## 6. Segmentação de clientes
--**Objetivo:** Organizar clientes em grupos estratégicos para ações de marketing direcionadas.

*Criar campanhas específicas, promoções personalizadas e fidelizar clientes estratégicos.*

```sql
    WITH rfm AS (
    SELECT
    cliente_id,
    nome_cliente,
    SUM(total_venda) AS valor_total,
    COUNT(venda_id) AS frequencia,
    DATE_DIFF(CURRENT_DATE(), MAX(data_venda), DAY) AS recencia
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    GROUP BY cliente_id, nome_cliente
    )
    SELECT
    cliente_id,
    nome_cliente,
    valor_total,
    frequencia,
    recencia,
    CASE
    WHEN valor_total >= 150 THEN 'VIP'
    WHEN valor_total >= 75 THEN 'Regular'
    ELSE 'Ocasional'
    END AS segmento
    FROM rfm
    ORDER BY valor_total DESC;

````

## 7. Previsão simples
--Média móvel 3 meses

--**Objetivo:** Criar estimativa de tendências futuras de vendas para planejamento de estoque e promoções.

*Fornece uma projeção simples da receita futura, ajudando na tomada de decisão sobre estoque, marketing e promoções.*

```sql

    WITH receita_mensal AS (
    SELECT
    FORMAT_DATE("%Y-%m", data_venda) AS ano_mes,
    SUM(total_venda) AS receita_total
    FROM `Livraria_DevSaber_1_10.vw_vendas_detalhes`
    GROUP BY ano_mes
    )
    SELECT
    ano_mes,
    receita_total,
    AVG(receita_total) OVER (ORDER BY ano_mes ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS media_movel_3meses
    FROM receita_mensal
    ORDER BY ano_mes;